# Full Setup: Normalize CSV, Load into SQLite, and Query with LangChain

In [ ]:
import pandas as pd
import sqlite3
import os
from langchain.chains import create_sql_query_chain
from langchain.sql_database import SQLDatabase
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Step 1: Loading CSV

In [ ]:
csv_data = {
    "restaurant": ["Cafe Spice", "Cafe Spice"],
    "category": ["Dessert", "Main Course"],
    "item_name": ["Chocolate Cake", "Butter Chicken"],
    "description": ["Rich chocolate cake", "Creamy spiced chicken"],
    "options": [None, "Rice/Naan"],
    "tags": ["gluten-free", "spicy, dairy"],
    "address": ["123 Main St", "123 Main St"],
    "state": ["NY", "NY"],
    "contact_info": ["1234567890", "1234567890"]
}
df = pd.DataFrame(csv_data)

# Step 2: Normalize & Store in SQLite

In [ ]:
db_name = "normalized_restaurant_data.db"
if os.path.exists(db_name):
    os.remove(db_name)
conn = sqlite3.connect(db_name)
cur = conn.cursor()

cur.executescript("""
CREATE TABLE IF NOT EXISTS restaurants (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE,
    contact_info TEXT
);

CREATE TABLE IF NOT EXISTS locations (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    restaurant_id INTEGER,
    address TEXT,
    state TEXT,
    FOREIGN KEY (restaurant_id) REFERENCES restaurants(id)
);

CREATE TABLE IF NOT EXISTS categories (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE
);

CREATE TABLE IF NOT EXISTS menu_items (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    restaurant_id INTEGER,
    category_id INTEGER,
    item_name TEXT,
    description TEXT,
    options TEXT,
    tags TEXT,
    FOREIGN KEY (restaurant_id) REFERENCES restaurants(id),
    FOREIGN KEY (category_id) REFERENCES categories(id)
);
""")

def get_or_create_id(table, column, value):
    cur.execute(f"SELECT id FROM {table} WHERE {column} = ?", (value,))
    result = cur.fetchone()
    if result:
        return result[0]
    cur.execute(f"INSERT INTO {table} ({column}) VALUES (?)", (value,))
    return cur.lastrowid

for _, row in df.iterrows():
    cur.execute("SELECT id FROM restaurants WHERE name = ?", (row["restaurant"],))
    res = cur.fetchone()
    restaurant_id = res[0] if res else cur.execute(
        "INSERT INTO restaurants (name, contact_info) VALUES (?, ?)",
        (row["restaurant"], row["contact_info"])).lastrowid

    cur.execute("""SELECT id FROM locations WHERE restaurant_id = ? AND address = ? AND state = ?""",
                (restaurant_id, row["address"], row["state"]))
    if not cur.fetchone():
        cur.execute("""INSERT INTO locations (restaurant_id, address, state) VALUES (?, ?, ?)""",
                    (restaurant_id, row["address"], row["state"]))

    category_id = get_or_create_id("categories", "name", row["category"])

    cur.execute("""INSERT INTO menu_items 
        (restaurant_id, category_id, item_name, description, options, tags) 
        VALUES (?, ?, ?, ?, ?, ?)""",
        (restaurant_id, category_id, row["item_name"], row["description"], row["options"], row["tags"]))

conn.commit()
conn.close()

# Step 3: Load SQLite DB for LangChain 

In [ ]:
db = SQLDatabase.from_uri("sqlite:///normalized_restaurant_data.db")

# Step 4: Setup Local HuggingFace Model 

In [ ]:
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

hf_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    temperature=0.3
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Step 5: Create SQL Chain 

In [ ]:
chain = create_sql_query_chain(llm, db)

# --- Step 6: Ask a Question ---

In [ ]:
question = "What desserts are available at Cafe Spice?"
response = chain.invoke({"question": question})

print("SQL Query Used:\n", response['intermediate_steps'][0])
print("\nAnswer:\n", response['result'])